In [ ]:
'''
My original Dataset
'''
'''
We create a dataset capable of loading the images in the nightowl dataset, then
a network to evaluate them and write the result in a json file with the right format
'''

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.nn import utils
import torchvision

from PIL import Image
import os
import json

from coco import COCO


'''
Custom Dataset in order to load the images with the bbox and the labels
'''
class MyCustomDataset(torch.utils.data.Dataset):
# __init__ function is where the initial logic happens like reading a csv
    def __init__(self, file_path, images_path):

        # Create the transformation that will be applied
        #self.transform_to_apply = transforms.Compose([ transforms.Resize(image_size),
            #transforms.CenterCrop(image_size), transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
        self.transform_to_apply = transforms.Compose([transforms.ToTensor()])
        
        self.images_path = images_path
        
        # get the dataset, the annotiations and the imagesID
        self.cocovar = COCO(file_path)
        self.imgIds = sorted(self.cocovar.getImgIds())

        self.imgs_count = len(self.imgIds)

    # __getitem__ function returns the data and labels. This function is called from dataloader
    # the format of return is a list containing the image with the applied transformation
    # and another list containing for every bbox [x,y,width,height,category_id]
    def __getitem__(self, index):

        # get the image id and its filename
        img_id = self.imgIds[index]
        cocoimg = self.cocovar.loadImgs(ids=img_id)[0]
        img_filename = cocoimg['file_name']

        # Open image and apply the transformation
        img_as_img = Image.open(os.path.join(self.images_path,img_filename))
        img = self.transform_to_apply(img_as_img)

        #load the ids of the annotations for the images
        image_annotations = self.cocovar.getAnnIds(img_id)
        
        anns_to_ret = []
        
        # for every annotation
        for ann_id in image_annotations:
            #load all the annotation data
            ann = self.cocovar.loadAnns(ids=ann_id)[0]
            data = []
            for i in ann['bbox']:
                data.append(i)
            data.append(ann['category_id'])
            # data will be: [x,y,width,height,category_id]
            
            anns_to_ret.append(data)

        return ( img, img_id)

    def __len__(self):
        return self.imgs_count

# Function that will create the dataset and the dataloader and return the dataloader
def getDataLoader(file_path,images_path,batch_size,num_workers):

    dataset = MyCustomDataset(file_path,images_path)

    data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)

    return data_loader

COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

def get_prediction(img_path, threshold,model):
    img = Image.open(img_path) # Load the image
    transform = transforms.Compose([transforms.ToTensor()]) # Defing PyTorch Transform
    img = transform(img) # Apply the transform to the image
    pred = model([img]) # Pass the image to the model
    pred_class = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list(pred[0]['labels'].numpy())] # Get the Prediction Score
    pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].detach().numpy())] # Bounding boxes
    pred_score = list(pred[0]['scores'].detach().numpy())
    pred_t = [pred_score.index(x) for x in pred_score if x > threshold][-1] # Get list of index with score greater than threshold.
    pred_boxes = pred_boxes[:pred_t+1]
    pred_class = pred_class[:pred_t+1]
    pred_scores =pred_score[:pred_t+1]
    return pred_boxes, pred_class,pred_scores     
    

def format_predictions(boxes,classes,scores,im_id):
    
    predictions = []
    
    for i in range(len(scores)):
        if(float(scores[i] < 0.3)):
            continue
        # take only the pedestrian
        cat_id = 1
        if(classes[i] != 1): #1=person in the COCO dataset
            continue
        
        x = {
            "image_id": int(im_id),
            "category_id": int(cat_id),
            "bbox":[round(float(boxes[i][0]),5), round(float(boxes[i][1]),5),
                    round(float(boxes[i][2] - boxes[i][0]),5) , round(float(boxes[i][3] -boxes[i][1]),5) ],
            "score": round(float(scores[i]),5)
            
        }
        predictions.append(x)
        
    return predictions
        
def format_output(output,ids):
    
    result = []
    
    for i in range( len(output) ):

        classes = output[i]['labels']
        scores = output[i]['scores']
        bboxes = output[i]['boxes']
        
        result.extend(format_predictions(bboxes,classes,scores,ids[i]))
        
    return result
            
        
def main():
    # Decide which device we want to run on
    device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
    print("Use device: " + str(device))
    
    batch_size = 4
    num_workers = 1

    file_path = '/home/test/data/nightowls/nightowls_validation.json'
    images_path = '/home/test/data/nightowls/nightowls_validation/'
    output_file = './test.json'
    
    #dataset = MyCustomDataset(file_path,images_path)
   
    # Load data
    data_loader = getDataLoader(file_path, images_path, batch_size, num_workers)
    
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    model.to(device)
    
    json_output = []
    
    for batch_index, (data,ids) in enumerate(data_loader,0):
        print('input:')
        print(str(ids.data.cpu().numpy()))
        dataaaa = data.to(device)
        #lbl = lbl.to(device)
        print('batch: ' + str(batch_index))
        outputs = model(dataaaa)
        
        json_output.extend(format_output(outputs,ids))
        
        with open(output_file, 'w') as f:
            json.dump(json_output, f) 
        
    
main()

In [ ]:
'''
Original PennFudanDataset
'''

import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image


class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

    def __getitem__(self, idx):
        # load images ad masks
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        mask = Image.open(mask_path)

        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [ ]:
'''
Training for the original PennFudanDataset
'''
from engine import train_one_epoch, evaluate
import utils
import transforms as T


def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

# use our dataset and defined transformations
dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))
dataset_test = PennFudanDataset('PennFudanPed', get_transform(train=False))

# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn, drop_last=True)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn, drop_last=True)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 2

# get the model using our helper function
model = get_instance_segmentation_model(num_classes)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)
        
# let's train it for 10 epochs
num_epochs = 3

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    #evaluate(model, data_loader_test, device=device)





In [1]:
'''
Try to modify my dataset to be more similar to the previous one
'''


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.nn import utils
import transforms as T
import torchvision

from PIL import Image
import os
import json

from coco import COCO

# annotation from the training set example:
# {"occluded":false,"difficult":false,"bbox":[775,207,53,140],"id":1003556,"category_id":1,"image_id":1004817,"pose_id":2,"tracking_id":1000185,"ignore":0,"area":7420,"truncated":false}
# bbox is [x,y,width,height]
# there are four category: [1,2,3,4]

'''
Custom Dataset in order to load the images with the bbox and the labels
'''
class MyCustomDataset(torch.utils.data.Dataset):
    def __init__(self, file_path, images_path, transforms=None):
        self.transforms = transforms
        
        self.images_path = images_path
        
        # get the dataset, the annotiations and the imagesID
        self.cocovar = COCO(file_path)
        self.imgIds = sorted(self.cocovar.getImgIds())

        self.imgs_count = len(self.imgIds)

    def __getitem__(self, index):
        
        # get the image id and its filename
        img_id = self.imgIds[index]
        cocoimg = self.cocovar.loadImgs(ids=img_id)[0]
        img_filename = cocoimg['file_name']

        # Open image and apply the transformation
        img_as_img = Image.open(os.path.join(self.images_path,img_filename))

        #load the ids of the annotations for the images
        image_annotations = self.cocovar.getAnnIds(img_id)
        
        boxes = []
        labels = []
        area = []
        occluded = []
        difficult = []
        pose_id = []
        ignore = []
        
        num_objs = len(image_annotations)
        
        
        # for every annotation
        for ann_id in image_annotations:
            #load all the annotation data
            ann = self.cocovar.loadAnns(ids=ann_id)[0]
            coordinates = []
            for i in ann['bbox']:
                coordinates.append(i)
            
            boxes.append(coordinates)
            labels.append(ann['category_id'])
            area.append(ann['area'])
            occluded.append(ann['occluded'])
            difficult.append(ann['difficult'])
            pose_id.append(ann['pose_id'])
            ignore.append(ann['ignore'])
            
        if(len(boxes)==0):
            boxes = [[10,10,100,100]]
            labels = [1]
        
            
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        image_id = torch.tensor([img_id])
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        
        img, target = self.transforms(img_as_img,target)
            
        #print('inside target: ' +str(target))
        

        return img, target

    def __len__(self):
        return self.imgs_count 

# Function that will create the dataset and the dataloader and return the dataloader
def getDataLoader(file_path,images_path,batch_size,num_workers):
    transforms = []
    transforms.append(T.ToTensor())
    transform = T.Compose(transforms)

    dataset = MyCustomDataset(file_path,images_path,transform)

    data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                              shuffle=True, num_workers=num_workers, drop_last=True,
                                              collate_fn=utils.collate_fn)

    return data_loader

'''
Import the model
'''
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

from engine import train_one_epoch, evaluate
import utils
        
def main(batch_size,num_workers,n_classes,num_epochs):

    device = torch.device("cuda:0")
    print("Use device: " + str(device))

    file_path = '/home/test/data/nightowls/nightowls_validation.json'
    images_path = '/home/test/data/nightowls/nightowls_validation/'
    output_file = './test.json'
   
    # Load data
    data_loader = getDataLoader(file_path, images_path, batch_size, num_workers)
    
    # Load the model
    model = get_instance_segmentation_model(n_classes)
    model.to(device)
    
    # Construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)

    # And a learning rate scheduler which decreases the learning rate by 10x every 30 epochs
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=30,gamma = 0.1)


    (imgs, targets) = next(iter(data_loader))
    print('targets: ' + str(targets))
    (imgs, targets) = next(iter(data_loader))
    print('targets: ' + str(targets))
    (imgs, targets) = next(iter(data_loader))
    print('targets: ' + str(targets))
    (imgs, targets) = next(iter(data_loader))
    print('targets: ' + str(targets))
    (imgs, targets) = next(iter(data_loader))
    print('targets: ' + str(targets))
    (imgs, targets) = next(iter(data_loader))
    print('targets: ' + str(targets))
    (imgs, targets) = next(iter(data_loader))
    print('targets: ' + str(targets))
    (imgs, targets) = next(iter(data_loader))
    print('targets: ' + str(targets))
    (imgs, targets) = next(iter(data_loader))
    print('targets: ' + str(targets))
    (imgs, targets) = next(iter(data_loader))
    print('targets: ' + str(targets))
    (imgs, targets) = next(iter(data_loader))
    print('targets: ' + str(targets))
    (imgs, targets) = next(iter(data_loader))
    print('targets: ' + str(targets))

    
    '''
    for i, (imgs,targets) in enumerate(data_loader):
        print('batch ' + str(i))
        print('targets: ' +str(targets))

        for t in targets:
            print('t:' + str(t) + ' type: ' + str(type(t)))
        
        #targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    '''

    # let's train it for some epochs
    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=1)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        #evaluate(model, data_loader_test, device=device)
    
    
        

batch_size = 1
num_workers = 1
n_classes = 4
num_epochs = 2
main(batch_size,num_workers,n_classes,num_epochs)



Use device: cuda:0
loading annotations into memory...
Done (t=0.24s)
creating index...
index created!
targets: ({'boxes': tensor([[ 10.,  10., 100., 100.]]), 'labels': tensor([1]), 'image_id': tensor([7026764])},)
targets: ({'boxes': tensor([[ 10.,  10., 100., 100.]]), 'labels': tensor([1]), 'image_id': tensor([7044543])},)
targets: ({'boxes': tensor([[344., 221.,  30.,  54.]]), 'labels': tensor([1]), 'image_id': tensor([7008421])},)
targets: ({'boxes': tensor([[ 10.,  10., 100., 100.]]), 'labels': tensor([1]), 'image_id': tensor([7010826])},)
targets: ({'boxes': tensor([[ 10.,  10., 100., 100.]]), 'labels': tensor([1]), 'image_id': tensor([7037861])},)
targets: ({'boxes': tensor([[ 10.,  10., 100., 100.]]), 'labels': tensor([1]), 'image_id': tensor([7007834])},)
targets: ({'boxes': tensor([[ 10.,  10., 100., 100.]]), 'labels': tensor([1]), 'image_id': tensor([7033338])},)
targets: ({'boxes': tensor([[ 10.,  10., 100., 100.]]), 'labels': tensor([1]), 'image_id': tensor([7019365])},)
ta

SystemExit: 1

/home/test/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
